# Importamos librerias

In [2]:
import pandas as pd
import numpy as np
import os
import snowflake.connector
import json
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
pd.options.mode.chained_assignment = None

In [3]:
while True:
    try:

        f = open('credentials.json')
        data_pass = json.load(f)

        pass_ = input("INGRESAR PASSCODE:")

        ctx = snowflake.connector.connect(
            user = data_pass['snow']['USER'],
            password = data_pass['snow']['PASS'],
            account = data_pass['snow']['ACCOUNT'],
            passcode = pass_,
            database = 'SANDBOX_PLUS',
            schema = 'DWH'
        )

        cursor = ctx.cursor()

        print('Connected')

        break

    except:
        print('Incorrect Password - provide again')

Connected


# Subclases

In [4]:
query = '''
SELECT
    DISTINCT
    SUB.SUB_NAME,
    SUB.SUBCLASE
FROM
    MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
WHERE
    LAA.ARTC_ARTC_COD IN ('259559000', '259559000', '250443000', '254878000', '240635000', '252351000', '259512000', '340343000')
    AND SUB.SUB_NAME NOT ILIKE '%NO%ESPECIFICADO%'
'''

cursor.execute(query)
subclases = cursor.fetch_pandas_all()
subclases.head(2)

,SUB_NAME,SUBCLASE
0,CUADRADA,500040002
1,YERBA TRADICIONAL,1600090003


# Segmento 5

In [42]:
# Clientes que en el ultimo mes no vengan a tata y tengan Score 5 en la subclase

In [10]:
tuple(subclases['SUBCLASE'].astype(str))

('500040002',
 '1600090003',
 '100020002',
 '1500030002',
 '2700020001',
 '1500050001')

In [25]:
query = '''
WITH VINIERON AS
    (
    SELECT
        DISTINCT SOCI_SOCI_ID
    FROM
        MSTRDB.DWH.FT_FDLN_MOVIMIENTOS
    WHERE
        GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
        AND TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE)
    )

SELECT
    SOCI_SOCI_ID,
    SUBCLASE,
    CORREO,
    CELULAR
FROM
    (
    SELECT
        LSS.SOCI_SOCI_ID,
        LSS.SUBCLASE,
        CONT.CORREO,
        CONT.CELULAR,
        ROW_NUMBER() OVER(PARTITION BY LSS.SOCI_SOCI_ID ORDER BY LSS.SOCI_SOCI_ID) AS R
    FROM
        BIZMETRIKS.DWH.LU_SCORING_SUBCLASE AS LSS
        INNER JOIN SANDBOX_PLUS.DWH.CONTACTABLES_2 AS CONT ON LSS.SOCI_SOCI_ID = CONT.SOCI_SOCI_ID
    WHERE
        LSS.SOCI_SOCI_ID NOT IN (SELECT * FROM VINIERON)
        AND LSS.SCORE >= 4
        AND LSS.SUBCLASE IN {subclases_snow}
    )
WHERE
    R = 1
'''

cursor.execute(query.format(subclases_snow = tuple(subclases['SUBCLASE'].astype(str))))
segm_5 = cursor.fetch_pandas_all()

segm_5 = segm_5.merge(subclases, on = 'SUBCLASE', how = 'left')

print(segm_5.shape[0])
segm_5.head(2)

150933


,SOCI_SOCI_ID,SUBCLASE,CORREO,CELULAR,SUB_NAME
0,2053073,100020002,joaquincocaro2019@gmail.com,92923983,CERVEZAS IMPORTADAS
1,654797,1500050001,lauraolaza75@gmail.com,99300560,ATUN


# Segmento 3

In [ ]:
# Para cada categoria (subclase) son los Clientes activos que no compran la subclase y tienen score 5

In [27]:
query = '''
WITH ACTIVOS AS
    (
    SELECT
        DISTINCT SOCI_SOCI_ID
    FROM
        MSTRDB.DWH.FT_FDLN_MOVIMIENTOS
    WHERE
        GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
        AND TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE)
    ),

COMPRAS_SUBCLASES AS
    (
    SELECT
        DISTINCT
        FFM.SOCI_SOCI_ID || ' - ' || IM.SUBCLASE AS KEY
    FROM
        MSTRDB.DWH.FT_FDLN_MOVIMIENTOS AS FFM
        INNER JOIN MSTRDB.DWH.FT_VENTAS AS FV ON FV.TICKET = FFM.TICKET
        INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
    WHERE
        FFM.GEOG_LOCL_ID IN (SELECT GEOG_LOCL_ID FROM MSTRDB.DWH.LU_GEOG_LOCAL WHERE GEOG_UNNG_ID = 2)
        AND FFM.TIEM_DIA_ID >= DATEADD(MONTH, -3, CURRENT_DATE)
    )


SELECT
    SOCI_SOCI_ID,
    SUBCLASE,
    CORREO,
    CELULAR
FROM
    (
    SELECT
        LSS.SOCI_SOCI_ID,
        LSS.SUBCLASE,
        CONT.CORREO,
        CONT.CELULAR,
        ROW_NUMBER() OVER(PARTITION BY LSS.SOCI_SOCI_ID ORDER BY LSS.SOCI_SOCI_ID) AS R
    FROM
        BIZMETRIKS.DWH.LU_SCORING_SUBCLASE AS LSS
        INNER JOIN SANDBOX_PLUS.DWH.CONTACTABLES_2 AS CONT ON LSS.SOCI_SOCI_ID = CONT.SOCI_SOCI_ID
    WHERE
        LSS.SOCI_SOCI_ID IN (SELECT * FROM ACTIVOS)
        AND LSS.SCORE = 5
        AND LSS.SUBCLASE IN {subclases_snow}
        AND LSS.SOCI_SOCI_ID || ' - ' || LSS.SUBCLASE NOT IN (SELECT * FROM COMPRAS_SUBCLASES)
    )
WHERE
    R = 1
'''

cursor.execute(query.format(subclases_snow = tuple(subclases['SUBCLASE'].astype(str))))
segm_3 = cursor.fetch_pandas_all()

segm_3 = segm_3.merge(subclases, on = 'SUBCLASE', how = 'left')

print(segm_3.shape[0])
segm_3.head(2)

164395


,SOCI_SOCI_ID,SUBCLASE,CORREO,CELULAR,SUB_NAME
0,313232,1600090003,None,None,YERBA TRADICIONAL
1,748034,1600090003,None,99745070,YERBA TRADICIONAL


# Excels

In [28]:
for i in segm_3['SUB_NAME'].unique():
    print(i)
    df_aux = segm_3[segm_3['SUB_NAME'] == i]
    df_aux.to_excel(f'Ofertata 4, S3 - {i}.xlsx', index = False)

YERBA TRADICIONAL
ATUN
CERVEZAS IMPORTADAS
A BASE DE TOMATE
DURAZNOS REGULARES
CUADRADA


In [29]:
for i in segm_5['SUB_NAME'].unique():
    print(i)
    df_aux = segm_5[segm_5['SUB_NAME'] == i]
    df_aux.to_excel(f'Ofertata 4, S5 - {i}.xlsx', index = False)

CERVEZAS IMPORTADAS
ATUN
A BASE DE TOMATE
YERBA TRADICIONAL
CUADRADA
DURAZNOS REGULARES


# Fin